# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [86]:
import pandas as pd
import sqlite3
from IPython.display import display

conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",)
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: 
BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, PNF_DATA, 


# REQUESTING USER'S ZIP TO COMPUTE PDP_REGION_CODE

In [87]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


# YIELDING ALL PLANS THAT ARE IN THE REGION AND HAVE S-CONTRACTS

In [88]:
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


# REQUESTING DRUG NAMES AND STRENGTHS FROM USER

In [89]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 3
Enter a generic drug name: METFORMIN
Enter a desired strength: 500
Enter a generic drug name: ALLOPURINOL
Enter a desired strength: 300
Enter a generic drug name: METOPROLOL SUCCINATE
Enter a desired strength: 50


{('ALLOPURINOL', '300'): [],
 ('METFORMIN', '500'): [],
 ('METOPROLOL SUCCINATE', '50'): []}

# YIELDING TABLES FOR EACH DRUG WITH REQUIRED DATA FOR COMPUTING COST WITH BENEFICIARY_COST_FILE (LONG)

In [90]:
#WARNING: THIS TAKES A LONG-ISH TIME
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql(
        " ".join(("select RXCUI_DATA.FULL_GENERIC_NAME, PLAN_INFO.FORMULARY_ID,\
                FORMULARY_DATA.RXCUI,FORMULARY_DATA.TIER_LEVEL_VALUE,\
                FORMULARY_DATA.PRIOR_AUTHORIZATION_YN,PLAN_INFO.CONTRACT_ID,\
                PLAN_INFO.PLAN_ID,PLAN_INFO.PLAN_NAME,PLAN_INFO.PREMIUM,PLAN_INFO.DEDUCTIBLE,\
                BENEFICIARY_COST.COVERAGE_LEVEL,BENEFICIARY_COST.DAYS_SUPPLY,\
                BENEFICIARY_COST.COST_AMT_PREF,BENEFICIARY_COST.COST_TYPE_PREF,\
                PLAN_INFO.PDP_REGION_CODE \
         from \
         FORMULARY_DATA join PLAN_INFO \
                        on substr(FORMULARY_DATA.FORMULARY_ID,4) = (PLAN_INFO.FORMULARY_ID) \
         join RXCUI_DATA on FORMULARY_DATA.RXCUI = RXCUI_DATA.RXCUI \
         join BENEFICIARY_COST on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                                  PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                                  cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                cast(BENEFICIARY_COST.TIER as integer) \
         where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
               PLAN_INFO.CONTRACT_ID like 'S%' and \
               upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and \
               RXCUI_DATA.STRENGTH like '"+strength+"%' and \
               (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
               BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
               cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1").split()),
        conn).drop_duplicates(subset='FORMULARY_ID',keep='first')

In [91]:
for key in user_generics:
    display(user_generics[key])

,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,24 HR Metformin hydrochloride 500 MG / saxagli...,18039,1043582,3,N,S5820,3,AARP MedicareRx Preferred (PDP),86.6,0.0,1.0,1,37.0,1.0,4
1,24 HR Metformin hydrochloride 500 MG / saxagli...,18040,1043582,3,N,S5921,349,AARP MedicareRx Saver Plus (PDP),33.2,405.0,1.0,1,30.0,1.0,4
2,24 HR Metformin hydrochloride 500 MG / saxagli...,18041,1043582,3,N,S0522,4,Symphonix Value Rx (PDP),30.4,405.0,1.0,1,35.0,1.0,4
3,24 HR Metformin hydrochloride 500 MG / saxagli...,18130,1043582,3,N,S5993,1,Horizon Medicare Blue Rx Standard (PDP),47.7,405.0,1.0,1,34.0,1.0,4
4,24 HR Metformin hydrochloride 500 MG / saxagli...,18131,1043582,3,N,S5993,3,Horizon Medicare Blue Rx Enhanced (PDP),99.6,0.0,1.0,1,32.0,1.0,4
6,24 HR Metformin hydrochloride 500 MG / sitagli...,18056,1243848,3,N,S5810,38,Aetna Medicare Rx Saver (PDP),34.9,325.0,1.0,1,30.0,1.0,4
7,24 HR Metformin hydrochloride 500 MG / sitagli...,18057,1243848,3,N,S5810,278,Aetna Medicare Rx Select (PDP),19.7,405.0,1.0,1,47.0,1.0,4
8,24 HR Metformin hydrochloride 500 MG / sitagli...,18059,1243848,3,N,S5768,127,First Health Part D Value Plus (PDP),56.2,0.0,1.0,1,47.0,1.0,4
9,24 HR Metformin hydrochloride 500 MG / sitagli...,18082,1243848,3,N,S5617,18,Cigna-HealthSpring Rx Secure (PDP),74.8,405.0,1.0,1,30.0,1.0,4
10,24 HR Metformin hydrochloride 500 MG / sitagli...,18083,1243848,3,N,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),52.9,0.0,1.0,1,42.0,1.0,4


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,Allopurinol 300 MG Oral Tablet,18039,197320,1,N,S5820,3,AARP MedicareRx Preferred (PDP),86.6,0.0,1.0,1,6.0,1.0,4
1,Allopurinol 300 MG Oral Tablet,18040,197320,1,N,S5921,349,AARP MedicareRx Saver Plus (PDP),33.2,405.0,1.0,1,1.0,1.0,4
2,Allopurinol 300 MG Oral Tablet,18041,197320,1,N,S0522,4,Symphonix Value Rx (PDP),30.4,405.0,1.0,1,1.0,1.0,4
3,Allopurinol 300 MG Oral Tablet,18043,197320,1,N,S5921,386,AARP MedicareRx Walgreens (PDP),26.8,405.0,0.0,1,0.0,1.0,4
5,Allopurinol 300 MG Oral Tablet,18056,197320,1,N,S5810,38,Aetna Medicare Rx Saver (PDP),34.9,325.0,0.0,1,1.0,1.0,4
7,Allopurinol 300 MG Oral Tablet,18057,197320,1,N,S5810,278,Aetna Medicare Rx Select (PDP),19.7,405.0,0.0,1,0.0,1.0,4
9,Allopurinol 300 MG Oral Tablet,18059,197320,1,N,S5768,127,First Health Part D Value Plus (PDP),56.2,0.0,1.0,1,1.0,1.0,4
10,Allopurinol 300 MG Oral Tablet,18082,197320,1,N,S5617,18,Cigna-HealthSpring Rx Secure (PDP),74.8,405.0,1.0,1,1.0,1.0,4
11,Allopurinol 300 MG Oral Tablet,18083,197320,1,N,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),52.9,0.0,1.0,1,4.0,1.0,4
12,Allopurinol 300 MG Oral Tablet,18130,197320,2,N,S5993,1,Horizon Medicare Blue Rx Standard (PDP),47.7,405.0,1.0,1,8.0,1.0,4


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE
0,24 HR metoprolol succinate 50 MG Extended Rele...,18039,866436,2,N,S5820,3,AARP MedicareRx Preferred (PDP),86.6,0.0,1.0,1,10.0,1.0,4
1,24 HR metoprolol succinate 50 MG Extended Rele...,18040,866436,1,N,S5921,349,AARP MedicareRx Saver Plus (PDP),33.2,405.0,1.0,1,1.0,1.0,4
2,24 HR metoprolol succinate 50 MG Extended Rele...,18041,866436,1,N,S0522,4,Symphonix Value Rx (PDP),30.4,405.0,1.0,1,1.0,1.0,4
3,24 HR metoprolol succinate 50 MG Extended Rele...,18043,866436,1,N,S5921,386,AARP MedicareRx Walgreens (PDP),26.8,405.0,0.0,1,0.0,1.0,4
5,24 HR metoprolol succinate 50 MG Extended Rele...,18056,866436,2,N,S5810,38,Aetna Medicare Rx Saver (PDP),34.9,325.0,0.0,1,2.0,1.0,4
7,24 HR metoprolol succinate 50 MG Extended Rele...,18057,866436,2,N,S5810,278,Aetna Medicare Rx Select (PDP),19.7,405.0,0.0,1,3.0,1.0,4
9,24 HR metoprolol succinate 50 MG Extended Rele...,18059,866436,2,N,S5768,127,First Health Part D Value Plus (PDP),56.2,0.0,1.0,1,2.0,1.0,4
10,24 HR metoprolol succinate 50 MG Extended Rele...,18082,866436,1,N,S5617,18,Cigna-HealthSpring Rx Secure (PDP),74.8,405.0,1.0,1,1.0,1.0,4
11,24 HR metoprolol succinate 50 MG Extended Rele...,18083,866436,1,N,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),52.9,0.0,1.0,1,4.0,1.0,4
12,24 HR metoprolol succinate 50 MG Extended Rele...,18130,866436,2,N,S5993,1,Horizon Medicare Blue Rx Standard (PDP),47.7,405.0,1.0,1,8.0,1.0,4


# COMPUTING COSTS ACROSS ROWS

In [92]:
for key in user_generics:
    user_generics[key]['TOTAL_COST'] = user_generics[key]['COST_AMT_PREF']*12 + \
                                        user_generics[key]['PREMIUM']*12 + \
                                        user_generics[key]['DEDUCTIBLE']
    user_generics[key] = user_generics[key].sort_values('TOTAL_COST',ascending=True)

In [93]:
top_five = {key:user_generics[key][:5] for key in user_generics}

In [94]:
for key in top_five:
    display(top_five[key])

,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE,TOTAL_COST
46,24 HR Metformin hydrochloride 500 MG Extended ...,18255,860975,1,N,S5884,150,Humana Walmart Rx Plan (PDP),20.4,405.0,0.0,1,1.0,1.0,4,661.8
20,Modified 24 HR Metformin hydrochloride 500 MG ...,18152,1807915,2,N,S5660,220,Express Scripts Medicare - Saver (PDP),22.6,405.0,0.0,1,4.0,1.0,4,724.2
29,24 HR Metformin hydrochloride 500 MG Extended ...,18043,860975,1,N,S5921,386,AARP MedicareRx Walgreens (PDP),26.8,405.0,0.0,1,0.0,1.0,4,726.6
45,24 HR Metformin hydrochloride 500 MG Extended ...,18254,860975,1,N,S5884,131,Humana Preferred Rx Plan (PDP),37.0,405.0,1.0,1,0.0,1.0,4,849.0
16,24 HR Metformin hydrochloride 500 MG / sitagli...,18419,1243848,3,N,S5601,8,SilverScript Choice (PDP),34.3,0.0,1.0,1,43.0,1.0,4,927.6


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE,TOTAL_COST
24,Allopurinol 300 MG Oral Tablet,18419,197320,2,N,S5601,8,SilverScript Choice (PDP),34.3,0.0,1.0,1,14.0,1.0,4,579.6
7,Allopurinol 300 MG Oral Tablet,18057,197320,1,N,S5810,278,Aetna Medicare Rx Select (PDP),19.7,405.0,0.0,1,0.0,1.0,4,641.4
20,Allopurinol 300 MG Oral Tablet,18255,197320,1,N,S5884,150,Humana Walmart Rx Plan (PDP),20.4,405.0,0.0,1,1.0,1.0,4,661.8
11,Allopurinol 300 MG Oral Tablet,18083,197320,1,N,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),52.9,0.0,1.0,1,4.0,1.0,4,682.8
9,Allopurinol 300 MG Oral Tablet,18059,197320,1,N,S5768,127,First Health Part D Value Plus (PDP),56.2,0.0,1.0,1,1.0,1.0,4,686.4


,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE,TOTAL_COST
24,24 HR metoprolol succinate 50 MG Extended Rele...,18419,866436,2,N,S5601,8,SilverScript Choice (PDP),34.3,0.0,1.0,1,14.0,1.0,4,579.6
7,24 HR metoprolol succinate 50 MG Extended Rele...,18057,866436,2,N,S5810,278,Aetna Medicare Rx Select (PDP),19.7,405.0,0.0,1,3.0,1.0,4,677.4
11,24 HR metoprolol succinate 50 MG Extended Rele...,18083,866436,1,N,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),52.9,0.0,1.0,1,4.0,1.0,4,682.8
20,24 HR metoprolol succinate 50 MG Extended Rele...,18255,866436,2,N,S5884,150,Humana Walmart Rx Plan (PDP),20.4,405.0,0.0,1,4.0,1.0,4,697.8
9,24 HR metoprolol succinate 50 MG Extended Rele...,18059,866436,2,N,S5768,127,First Health Part D Value Plus (PDP),56.2,0.0,1.0,1,2.0,1.0,4,698.4


# CHEAPEST COMBINATION OF DRUGS

In [109]:
cheapest_combo = pd.DataFrame()
for key in top_five:
    cheapest_combo = pd.concat([cheapest_combo,top_five[key][:1]],axis=0)

In [116]:
cheapest_combo

,FULL_GENERIC_NAME,FORMULARY_ID,RXCUI,TIER_LEVEL_VALUE,PRIOR_AUTHORIZATION_YN,CONTRACT_ID,PLAN_ID,PLAN_NAME,PREMIUM,DEDUCTIBLE,COVERAGE_LEVEL,DAYS_SUPPLY,COST_AMT_PREF,COST_TYPE_PREF,PDP_REGION_CODE,TOTAL_COST
46,24 HR Metformin hydrochloride 500 MG Extended ...,18255,860975,1,N,S5884,150,Humana Walmart Rx Plan (PDP),20.4,405.0,0.0,1,1.0,1.0,4,661.8
24,Allopurinol 300 MG Oral Tablet,18419,197320,2,N,S5601,8,SilverScript Choice (PDP),34.3,0.0,1.0,1,14.0,1.0,4,579.6
24,24 HR metoprolol succinate 50 MG Extended Rele...,18419,866436,2,N,S5601,8,SilverScript Choice (PDP),34.3,0.0,1.0,1,14.0,1.0,4,579.6


# FETCHING PHARMACIES

In [161]:
#takes a long-ish time because PNF_DATA table in database is GIGANTIC 
#(multiple gigabytes, requires load distributing server for speed)
pharms = {}
for i,med in cheapest_combo[['FULL_GENERIC_NAME','CONTRACT_ID','PLAN_ID']].iterrows():
    pharms[med['FULL_GENERIC_NAME']] = \
    pd.read_sql("select NCPDP.PHARMACY_NAME,NCPDP.PHONE,PNF_DATA.* \
    from PNF_DATA,NCPDP \
    where PNF_DATA.CONTRACT_ID is '"+med['CONTRACT_ID']+"' and \
    PNF_DATA.PLAN_ID is '"+"0"*(3-len(str(med['PLAN_ID'])))+str(med['PLAN_ID'])+"' and \
    PNF_DATA.PHARMACY_ZIPCODE is '"+user_zip+"' and \
    NCPDP.NPI is substr(PNF_DATA.PHARMACY_NUMBER,3)",conn)

In [164]:
for med,pharm in pharms.items():
    print(med)
    display(pharm)

24 HR Metformin hydrochloride 500 MG Extended Release Oral Tablet


,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,CVS PHARMACY# 10211,9737630124,S5884,150,000,101861888299,07040,N,N,Y,N,1,1,0,1,1.1,0,1.1
1,WALGREENS,9737617391,S5884,150,000,101174806756,07040,N,N,Y,N,1,1.2,0,1.2,1.2,0,1.2
2,TOWN AND COUNTRY PHARMACY,9737622700,S5884,150,000,101194716548,07040,N,N,Y,N,1,1,0,1,1,0,1
3,CAROLINA SUBURBAN DRUG AND SURGICAL,9737635252,S5884,150,000,101730197997,07040,N,N,Y,N,1,1,0,1,1,0,1
4,LEXANN PHARMACY & WELLNESS CENTER,9737630010,S5884,150,000,101780082339,07040,N,N,Y,N,1,1,0,1,1,0,1


Allopurinol 300 MG Oral Tablet


,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,WALGREENS,9737617391,S5601,008,000,101174806756,07040,N,N,Y,N,1,0.5,0.5,0.5,0.5,0.5,0.5
1,CVS PHARMACY# 10211,9737630124,S5601,008,000,101861888299,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
2,LEXANN PHARMACY & WELLNESS CENTER,9737630010,S5601,008,000,101780082339,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
3,TOWN AND COUNTRY PHARMACY,9737622700,S5601,008,000,101194716548,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
4,CAROLINA SUBURBAN DRUG AND SURGICAL,9737635252,S5601,008,000,101730197997,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4


24 HR metoprolol succinate 50 MG Extended Release Oral Tablet


,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,WALGREENS,9737617391,S5601,008,000,101174806756,07040,N,N,Y,N,1,0.5,0.5,0.5,0.5,0.5,0.5
1,CVS PHARMACY# 10211,9737630124,S5601,008,000,101861888299,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
2,LEXANN PHARMACY & WELLNESS CENTER,9737630010,S5601,008,000,101780082339,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
3,TOWN AND COUNTRY PHARMACY,9737622700,S5601,008,000,101194716548,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
4,CAROLINA SUBURBAN DRUG AND SURGICAL,9737635252,S5601,008,000,101730197997,07040,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
